In [38]:
import requests
import pandas as pd
import json
from tqdm import tqdm 
from time import sleep

In [22]:
with open ('./ufs.json', 'r', encoding='utf8') as f:
    ufs = json.load(f)

In [34]:
orgs_final = []
for uf in tqdm(ufs):
    u = uf.get('sigla')
    url = f"https://sgip3.tse.jus.br/sgip3-consulta/api/v1/orgaoPartidario/consulta?dataFimVigencia=31%2F10%2F2023&dataInicioVigencia=01%2F06%2F2017&isComposicoesHistoricas=true&nrZona=0&sgUe={u}&sqPartido=0&tpAbrangencia=82"
    while True:
        try:
            resp = requests.get(url)
            break
        except requests.exceptions.ConnectionError:
            sleep(5)
            print("sleeping")
    orgs = resp.json()
    orgs_final += orgs

  0%|          | 0/26 [00:00<?, ?it/s]

100%|██████████| 26/26 [00:43<00:00,  1.68s/it]


In [35]:
pd.DataFrame(orgs_final).to_excel('orgaos.xlsx')

In [41]:
membros_final = []
for org in tqdm(orgs_final):
    url_org = f"https://sgip3.tse.jus.br/sgip3-consulta/api/v1/orgaoPartidario/comAnotacoesEMembros?idOrgaoPartidario={org.get('sqOrgaoPartidario')}"
    while True:
        try:
            resp = requests.get(url_org)
            break
        except requests.exceptions.ConnectionError:
            print('sleeping')
            sleep(5)
    membros = resp.json().get('membros')
    membros_final += membros

  0%|          | 15/5480 [00:07<43:27,  2.10it/s]

sleepinig


  0%|          | 20/5480 [00:14<1:14:34,  1.22it/s]

sleepinig


  1%|          | 45/5480 [00:34<54:46,  1.65it/s]  

sleepinig


  1%|          | 62/5480 [00:51<1:14:17,  1.22it/s]


KeyboardInterrupt: 

In [29]:
membros['membros']

[{'id': 9513580,
  'nomeMembro': 'MARCOS FELIPE SILVA VIEGAS',
  'nomeCargo': 'SUPLENTE DA COMISSÃO EXECUTIVA REGIONAL',
  'dataInicioExercicio': '14/07/2023',
  'dataFimExercicio': '14/07/2023',
  'situacao': 'Inativo',
  'respAdministrativo': 'NÃO',
  'respFinanceiro': 'NÃO',
  'nrTitulo': '005767612402',
  'cpf': '00135757240'},
 {'id': 9513578,
  'nomeMembro': 'FRANCISCO ELINALDO MESQUITA DA SILVA',
  'nomeCargo': 'PRESIDENTE DA COMISSÃO DIRETORA PROVISÓRIA REGIONAL',
  'dataInicioExercicio': '14/07/2023',
  'dataFimExercicio': '14/07/2023',
  'situacao': 'Inativo',
  'respAdministrativo': 'SIM',
  'respFinanceiro': 'SIM',
  'nrTitulo': '003077782402',
  'cpf': '50919520200'},
 {'id': 9513579,
  'nomeMembro': 'JAIRO NASCIMENTO FERREIRA',
  'nomeCargo': 'VICE- PRESIDENTE DA COMISSÃO DIRETORA PROVISÓRIA REGIONAL',
  'dataInicioExercicio': '14/07/2023',
  'dataFimExercicio': '14/07/2023',
  'situacao': 'Inativo',
  'respAdministrativo': 'NÃO',
  'respFinanceiro': 'NÃO',
  'nrTitulo': 